In [165]:
import pandas as pd
import numpy as np


In [166]:
#read file, sheet_name=None to read multiple sheets
dict_df = pd.read_excel("../data/raw/Moving annual rent by suburb - March Quarter 2021.xlsx",sheet_name=None,header=None)
#Source of Hisotrical rental data: https://www.dhhs.vic.gov.au/past-rental-reports
#Rental Report 2021: Moving annualrents by suburb - March

In [167]:
#Define function - gets the year of the last four bits of a string
def get_year(s):
    try:
        return int(s[-4:])
    except:
        return np.nan

In [168]:
def get_month(s):
    try:
        return s[:3]
    except:
        return np.nan

In [169]:
is_first =True
for sheet in dict_df:
    if is_first:
        df = dict_df[sheet].iloc[2:4,:].copy() #header
        df = df.fillna(method="pad",axis=1)#Fill in the missing values with the previous column
        is_first = False
    df_sheet = dict_df[sheet].iloc[4:,].copy() 
    df_sheet["types"] = sheet #Adding a 'type' column
    df = df.append(df_sheet) #merge to the main table
del df[0]

/tmp/ipykernel_16840/3567915256.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_sheet) #merge to the main table
/tmp/ipykernel_16840/3567915256.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_sheet) #merge to the main table
/tmp/ipykernel_16840/3567915256.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_sheet) #merge to the main table
/tmp/ipykernel_16840/3567915256.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_sheet) #merge to the main table
/tmp/ipykernel_16840/3567915256.py:9: FutureWarning: The frame.append method is deprecated and will be remov

In [170]:
df_T = df.T # Transpose in numpy, for logical indexing
df_T["year"] = df_T.iloc[:,0].apply(get_year) #apply (get_year) to obtain column 'year'
df_T["month"] = df_T.iloc[:,0].apply(get_month) #apply (get_month) to obtain column 'month'
df_T = df_T.loc[((df_T["year"]>=2016) & (df_T["year"]<=2021)&(df_T["month"]=="Jun"))|df_T["year"].isna() ] #Keep rows with year of 2016-2021 and null values
df_T = df_T.loc[(df_T[3] == "Median" )|df_T[3].isna() ] #keep median column
del df_T[3],df_T[2] #remove unnecessary columns
del df_T["month"]#remove month column
del df_T["year"]#remove year column
df_all = df_T.T #transpose
df_all.columns = ["Sub",2016,2017,2018,2019,2020,"Type"] #Set the column name

In [171]:
df_all

,Sub,2016,2017,2018,2019,2020,Type
4,Albert Park-Middle Park-West St Kilda,325,340,350,350,355,1 bedroom flat
5,Armadale,310,325,395,360,368,1 bedroom flat
6,Carlton North,330,345,348,370,360,1 bedroom flat
7,Carlton-Parkville,330,355,375,390,356,1 bedroom flat
8,CBD-St Kilda Rd,380,400,420,434,425,1 bedroom flat
...,...,...,...,...,...,...,...
158,Wanagaratta,260,265,270,290,300,All properties
159,Warragul,290,300,320,350,350,All properties
160,Warrnambool,290,290,300,320,340,All properties
161,Wodonga,290,300,300,320,330,All properties


In [172]:
def find_sub(n,row):
    sub=row['Sub']
    sublist=['','','']
    currsub=''
    for character in sub:
        if character != '-':
            currsub += character
        else:
            sublist.append(currsub)
            currsub=''
    sublist.append(currsub)
    return sublist[n]



In [173]:
df_all['sub1'] = df_all.apply(lambda row: find_sub(-1,row), axis=1)
df_all['sub2'] = df_all.apply(lambda row: find_sub(-2,row), axis=1)
df_all['sub3'] = df_all.apply(lambda row: find_sub(-3,row), axis=1)
df_all['sub4'] = df_all.apply(lambda row: find_sub(-4,row), axis=1)
del df_all['Sub']

In [174]:
df_all

,2016,2017,2018,2019,2020,Type,sub1,sub2,sub3,sub4
4,325,340,350,350,355,1 bedroom flat,West St Kilda,Middle Park,Albert Park,
5,310,325,395,360,368,1 bedroom flat,Armadale,,,
6,330,345,348,370,360,1 bedroom flat,Carlton North,,,
7,330,355,375,390,356,1 bedroom flat,Parkville,Carlton,,
8,380,400,420,434,425,1 bedroom flat,St Kilda Rd,CBD,,
...,...,...,...,...,...,...,...,...,...,...
158,260,265,270,290,300,All properties,Wanagaratta,,,
159,290,300,320,350,350,All properties,Warragul,,,
160,290,290,300,320,340,All properties,Warrnambool,,,
161,290,300,300,320,330,All properties,Wodonga,,,


In [175]:
df1 = df_all[[2016, 2017, 2018, 2019, 2020, 'Type', 'sub1']].rename(columns={'sub1': 'Sub'})
df2 = df_all[[2016, 2017, 2018, 2019, 2020, 'Type', 'sub2']].rename(columns={'sub2': 'Sub'})
df3 = df_all[[2016, 2017, 2018, 2019, 2020, 'Type', 'sub3']].rename(columns={'sub3': 'Sub'})
df4 = df_all[[2016, 2017, 2018, 2019, 2020, 'Type', 'sub4']].rename(columns={'sub4': 'Sub'})

In [176]:
df_all

,2016,2017,2018,2019,2020,Type,sub1,sub2,sub3,sub4
4,325,340,350,350,355,1 bedroom flat,West St Kilda,Middle Park,Albert Park,
5,310,325,395,360,368,1 bedroom flat,Armadale,,,
6,330,345,348,370,360,1 bedroom flat,Carlton North,,,
7,330,355,375,390,356,1 bedroom flat,Parkville,Carlton,,
8,380,400,420,434,425,1 bedroom flat,St Kilda Rd,CBD,,
...,...,...,...,...,...,...,...,...,...,...
158,260,265,270,290,300,All properties,Wanagaratta,,,
159,290,300,320,350,350,All properties,Warragul,,,
160,290,290,300,320,340,All properties,Warrnambool,,,
161,290,300,300,320,330,All properties,Wodonga,,,


In [177]:
newdf_all=pd.concat([df1, df2,df3,df4], axis=0)

In [178]:
newdf_all=newdf_all[newdf_all['Sub'] != '']


In [ ]:
def changeCBDname(row):
    sub=row['Sub']
    if sub == 'CBD':
        return 'Melbourne'
    return sub

In [ ]:
newdf_all['Sub'] = newdf_all.apply(lambda row: changeCBDname(row), axis=1)

In [179]:
is_first = True
for year in range(2016,2021):
    df_year = newdf_all[["Sub","Type"]].copy() 
    df_year["Year"] = year
    df_year["Median_Price"] =  newdf_all[year].copy()
    if is_first:
        df_result = df_year.copy()
        is_first = False
    else:
        df_result = df_result.append(df_year)

/tmp/ipykernel_16840/78783640.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(df_year)
/tmp/ipykernel_16840/78783640.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(df_year)
/tmp/ipykernel_16840/78783640.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(df_year)
/tmp/ipykernel_16840/78783640.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(df_year)


In [180]:
df_result

,Sub,Type,Year,Median_Price
4,West St Kilda,1 bedroom flat,2016,325
5,Armadale,1 bedroom flat,2016,310
6,Carlton North,1 bedroom flat,2016,330
7,Parkville,1 bedroom flat,2016,330
8,St Kilda Rd,1 bedroom flat,2016,380
...,...,...,...,...
56,Mentone,All properties,2020,435
81,Oak Park,All properties,2020,400
86,Bundoora,All properties,2020,400
87,Eltham,All properties,2020,450


In [181]:
#output to csv
df_result.to_csv("../data/curated/historical_data.csv",index=False)